# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БВТ2005 Тимошин Алексей Витальевич
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

Импорт библиотек

In [1]:
import pandas as pd
from pyvis.network import Network
import networkx as nx
import math
import copy
import random
import time

Алгоритм генерации случайного графа в виде матрицы смежности

In [2]:
def generate_matrix():
    size = random.randint(30, 100)
    matrix = [[0 for i in range(size)] for j in range(size)]
    for i in range(size):
        for j in range(size):
            if random.randint(0, 3) == 3:
                rand_count = random.randint(1, 20)
                matrix[i][j] = rand_count
    return matrix

Настройки для графического отображения графов

In [3]:
options = """var options = {
  "edges": {
    "arrows": {
      "to": {
        "enabled": true
      }
    },
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "interaction": {
    "navigationButtons": true
  },
  "physics": {
    "minVelocity": 0.75
  }
}
"""

In [4]:
#создание графа, используя матрицу смежности
def create_graph(matrix : list) -> nx.DiGraph:
    graph = nx.DiGraph()
    for i in range(1, len(matrix)):
        graph.add_node(i)
    #graph.add_edges_from(edges_dict)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] != 0 and i != j:
                graph.add_edge(i + 1, j + 1, weight=matrix[i][j], label=matrix[i][j])
    print(graph.edges())
    for edge in graph.edges():
        graph.edges[edge]['color'] = 'blue'
    return graph

Алгоритм Флойда-Уоршелла

In [5]:
#загрузка матрицы из файла
with open("mat2.txt", "r", encoding = "utf-8") as file:
    matrix = [[int(num) for num in line.split(" ")] for line in file]

In [6]:
#создаём граф
graph = create_graph(matrix)


#отображаем граф из файла
nt = Network(notebook = True)
nt.from_nx(graph)
nt.set_options(options)
nt.show("test.html")

[(1, 2), (1, 3), (2, 3), (2, 6), (3, 4), (3, 5), (3, 6), (4, 2), (4, 5), (4, 6), (5, 3), (5, 4), (5, 6), (6, 2), (6, 4)]


In [7]:
#получение пути
def get_path(H, v, u):
    v -= 1
    u -= 1
    result = []
    result.append(v)
    while v != u:
        v = H[v][u]
        result.append(v)
    __result = [el + 1 for el in result]
    return __result


#алгоритм
def floyd(matrix):
    W = copy.deepcopy(matrix)
    N = len(W)
    H = [[0 for i in range(N)] for j in range(N)]
    for i in range(len(W)):
        for j in range(len(W)):
            if W[i][j] == 0:
                W[i][j] = math.inf
            else:
                H[i][j] = j
    for i in range(len(W)):
        for j in range(len(W)):
            if i == j: continue
            if W[i][j] != math.inf:
                for k in range(len(W)):
                    if i == k or j == k: continue
                    if W[i][k] > W[i][j] + W[j][k]:
                        W[i][k] = W[i][j] + W[j][k]
                        H[i][k] = H[i][j]
    return (W, H)


result = floyd(matrix)
path = get_path(result[1], 2, 5)


#вносим графические изменения
graph1 = copy.deepcopy(graph)
for i in range(1, len(path)):
    graph1.nodes[path[i - 1]]['color'] = 'red'
    graph1.edges[path[i - 1], path[i]]['color'] = 'red'
graph1.nodes[path[-1]]['color'] = 'red'


#отображаем граф с решением
nt1 = Network(notebook=True)
nt1.set_options(options)
nt1.from_nx(graph1)
nt1.show("test1.html")

In [8]:
#тестовый блок для определения скорости работы алгоритма

tests = []

for i in range(20):
    temp_matrix = generate_matrix()
    node_counts = len(temp_matrix)
    edge_counts = 0
    for i in range(len(temp_matrix)):
        for j in range(len(temp_matrix)):
            if temp_matrix[i][j] != 0:
                edge_counts += 1
    t1 = time.time()
    result = floyd(temp_matrix)
    t2 = time.time()
    delta = round((t2 - t1) * 1000)
    tests.append([node_counts, edge_counts, delta])

pd.DataFrame(tests, columns = ["Количество узлов", "Количество ребер", "Время в мс"])

,Количество узлов,Количество ребер,Время в мс
0,55,775,38
1,78,1474,103
2,57,802,40
3,78,1487,102
4,31,236,7
5,72,1289,82
6,53,686,33
7,36,304,10
8,45,511,20
9,38,362,12


Алгоритм Дейкстры

In [9]:
#загрузка матрицы из файла
with open("mat2.txt", "r", encoding = "utf-8") as file:
    matrix = [[int(num) for num in line.split(" ")] for line in file]

In [10]:
#создаём граф
graph = create_graph(matrix)


#отображаем граф из файла
nt = Network(notebook = True)
nt.from_nx(graph)
nt.set_options(options)
nt.show("test.html")

[(1, 2), (1, 3), (2, 3), (2, 6), (3, 4), (3, 5), (3, 6), (4, 2), (4, 5), (4, 6), (5, 3), (5, 4), (5, 6), (6, 2), (6, 4)]


In [11]:
#получение пути
def get_path(P, v, u):
    result = []
    result.append(u)
    while u != v:
        u = P[u]
        result.append(u)
    __result = [el + 1 for el in result]
    return __result[::-1]


#алгоритм
def dijkstra(matrix, v, u):
    v -= 1
    u -= 1
    N = len(matrix)
    for i in range(N):
        for j in range(N):
            if matrix[i][j] == 0 and i != j:
                matrix[i][j] = math.inf
    D = [matrix[v][i] for i in range(N)]
    P = [v for i in range(N)]
    checked_nodes = {v}
    for i in range(0, N):
        temp_min = math.inf
        node = None
        for i in range(N):
            if D[i] < math.inf and i not in checked_nodes:
                temp_min = D[i]
                node = i
        if node is not None:
            checked_nodes.add(node)
            for i in range(N):
                if D[i] > matrix[node][i] + D[node]:
                    D[i] = matrix[node][i] + D[node]
                    P[i] = node
    if D[u] == math.inf:
        path = []
    else:
        path = get_path(P, v, u)
    return (D, path)



result = dijkstra(matrix, 1, 6)
path = result[1]



#вносим графические изменения

graph1 = copy.deepcopy(graph)
if len(path) > 0:
    for i in range(1, len(path)):
        graph1.nodes[path[i - 1]]['color'] = 'red'
        graph1.edges[path[i - 1], path[i]]['color'] = 'red'
    graph1.nodes[path[-1]]['color'] = 'red'



#отображаем граф с решением
nt1 = Network(notebook=True)
nt1.set_options(options)
nt1.from_nx(graph1)
nt1.show("test1.html")

In [12]:
#тестовый блок для определения скорости работы алгоритма

tests = []

for i in range(20):
    temp_matrix = generate_matrix()
    node_counts = len(temp_matrix)
    edge_counts = 0
    for i in range(len(temp_matrix)):
        for j in range(len(temp_matrix)):
            if temp_matrix[i][j] != 0:
                edge_counts += 1
    t1 = time.time()
    c1 = random.randint(0, len(temp_matrix) // 2)
    c2 = random.randint(0, len(temp_matrix))
    result = dijkstra(temp_matrix, c1, c2)
    t2 = time.time()
    delta = round((t2 - t1) * 1000)
    tests.append([node_counts, edge_counts, delta])

pd.DataFrame(tests, columns = ["Количество узлов", "Количество ребер", "Время в мс"])

,Количество узлов,Количество ребер,Время в мс
0,52,642,1
1,66,1110,2
2,36,351,1
3,93,2119,4
4,51,679,1
5,92,2144,4
6,63,994,1
7,78,1536,2
8,56,782,1
9,47,555,0


Алгоритм Беллмана-Форда

In [ ]:
#загрузка матрицы из файла
with open("mat1.txt", "r", encoding = "utf-8") as file:
    matrix = [[int(num) for num in line.split(" ")] for line in file]

In [ ]:
#создаём граф
graph = nx.Graph()
for i in range(len(matrix)):
    graph.add_node(i + 1, size = 10)
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        if matrix[i][j] != 0:
            graph.add_edge(i + 1, j + 1, weight = matrix[i][j], color = 'blue')

#графически отоброжаем наш изначальный граф
nt = Network(notebook = True)
nt.from_nx(graph)
nt.show("test.html")

In [ ]:
#сам алгоритм

def get_path(P, v, u):
    N = len(P)
    j = P[N - 1][u]
    result = []
    result.append(u)
    result.append(j)
    for i in range(N - 1, -1, -1):
        j = P[i - 1][j]
        if j == None:
            break
        result.append(j)
    __result = [el + 1 for el in result]
    return __result[::-1]

def bell_ford(matrix, v, u):
    v -= 1
    u -= 1
    w = {}
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] != 0:
                w[(i, j)] = matrix[i][j]
    N = len(matrix)
    A = [[math.inf for i in range(N)] for j in range(N)]
    A[0][v] = 0
    P = [[None for i in range(N)] for j in range(N)]
    for k in range(1, N):
        flag = True
        for i in range(N):
            A[k][i] = A[k - 1][i]
            for j in range(N):
                if A[k - 1][j] + matrix[j][i] < A[k][i] and matrix[j][i] != 0:
                    A[k][i] = A[k - 1][j] + matrix[j][i]
                    flag = False
                    P[k][i] = j
        if flag:
            break
    i = N - 1
    while i > -1:
        if A[i][v] == math.inf:
            A.pop(i)
            i -= 1
        else:
            break
    i = N - 1
    while i > -1:
        if P[i][u] == None:
            P.pop(i)
            i -= 1
        else:
            break
    return get_path(P, v, u)
new_matrix = copy.deepcopy(matrix)
result = bell_ford(matrix, 1, 8)


#вносим графические изменения
graph1 = copy.deepcopy(graph)
for i in range(1, len(result)):
    graph1.nodes[result[i - 1]]['color'] = 'red'
    graph1.nodes[result[i]]['color'] = 'red'
    graph1.edges[result[i], result[i - 1]]['color'] = 'red'
    graph1.edges[result[i], result[i - 1]]['with_label'] = True
    graph1.edges[result[i], result[i - 1]]['label'] = graph1.edges[result[i], result[i - 1]]['weight']


#отображаем граф с решением
nt1 = Network(notebook = True)
nt1.from_nx(graph1)
nt1.show("test1.html")

In [ ]:
#тестовый блок для определения скорости работы алгоритма

tests = []

for i in range(10):
    temp_matrix = generate_matrix()
    node_counts = len(temp_matrix)
    edge_counts = 0
    for i in range(len(temp_matrix)):
        for j in range(i):
            if temp_matrix[j][i] != 0:
                edge_counts += 1
    t1 = time.time()
    c1 = random.randint(1, len(temp_matrix) // 2)
    c2 = random.randint(1, len(temp_matrix))
    result = bell_ford(temp_matrix, c1, c2)
    t2 = time.time()
    delta = round((t2 - t1) * 1000)
    tests.append([node_counts, edge_counts, delta])

pd.DataFrame(tests, columns = ["Количество узлов", "Количество ребер", "Время в мс"])

Алгоритм Джонсона

In [ ]:
#загрузка матрицы из файла
with open("mat1.txt", "r", encoding = "utf-8") as file:
    matrix = [[int(num) for num in line.split(" ")] for line in file]

In [ ]:
#создаём граф
graph = nx.Graph()
for i in range(len(matrix)):
    graph.add_node(i + 1, size = 10)
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        if matrix[i][j] != 0:
            graph.add_edge(i + 1, j + 1, weight = matrix[i][j], color = 'blue')

#графически отоброжаем наш изначальный граф
nt = Network(notebook = True)
nt.from_nx(graph)
nt.show("test.html")

In [ ]:
#сам алгоритм


Алгоритм Левита

Алгоиртм Йена

### Вывод